In [2]:
# !pip install beautifulsoup4 lxml
# !pip install requests


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [48]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

data = []
df = pd.DataFrame(columns=['TEMP', 'HTS6', 'HTS_9', 'DESCRIPTION'])


def loop_the_page(page):
    page_2dg = str(page).zfill(2)
    
    url = f"https://www.customs.go.jp/english/tariff/2023_04_01/data/e_{page_2dg}.htm"

    response = requests.get(url)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, "html.parser")

    table = soup.find('table', class_='t', attrs={'name': 'TV'})
    return table






In [49]:
try:
    for page in range(1, 98):
        table = loop_the_page(page)
        if table:
            # Extract and print the content of the table
            # print(table.prettify())
            
            for row in table.find_all('tr', id= 'tr'):
                cols = row.find_all('td')
                col_data = [col.get_text(strip=True) for col in cols]
                # Append data to the DataFrame
                df = pd.concat([df, pd.DataFrame([col_data], columns=['TEMP', 'HTS6', 'HTS_9', 'DESCRIPTION'])], ignore_index=True)
        else:
            print("Table not found.")
except:
    pass


# for row in table.find_all('tr', class_= 'td'):
#     print(row)
    # cols = row.find_all('td', class_='td')
    # col_data = [col.get_text(strip = TRUE) for col in cols]
    # data.append(col_data)
    
# df = pd.DataFrame(data, columns=['Col1', 'Col2', 'Col3'])

# print(df)

# for paragraph in soup.find_all("p"):
#     print(paragraph.text)


In [53]:
# print(df)
df_new = df[["HTS6", "HTS_9", "DESCRIPTION"]]

df_new

# df.to_csv("japan_hts_all_digits.csv")

,HTS6,HTS_9,DESCRIPTION
0,01.01,,"Live horses, asses, mules and hinnies"
1,,,Horses :
2,0101.21,,Pure-bred breeding animals
3,,100,1 Certified as being those other than Thorough...
4,,,2 Other
...,...,...,...
12757,76.16,,Other articles of aluminium
12758,7616.10,000,"Nails, tacks, staples (other than those of hea..."
12759,,,Other :
12760,7616.91,000,"Cloth, grill, netting and fencing, of aluminiu..."


In [104]:
pre_num6 = "000000"
pre_num_9 = "000"
for key, values in df_new.iterrows():
    if values.HTS6.replace(".","").zfill(6) != "000000":
        # print(values.HTS6.replace(".","").zfill(6))
        df_new.loc[key, "HTS6"] = values.HTS6.replace(".","").zfill(6)
        pre_num6 = values.HTS6.replace(".","").zfill(6)
    else:
        # print(pre_num6)
        df_new.loc[key, "HTS6"] = pre_num6
        
    if values.HTS_9 == '000':
        # print(values.HTS_9)
        df_new.loc[key, "HTS_9"] = values.HTS_9
        pre_num_9 = values.HTS_9
    elif values.HTS_9 != '':     
        # print(values.HTS_9)
        df_new.loc[key, "HTS_9"] = values.HTS_9
        pre_num_9 = values.HTS_9
    else:
        # print(pre_num_9)
        df_new.loc[key, "HTS_9"] = pre_num_9
    
    df_new.loc[key, "DESCRIPTION"] = values.DESCRIPTION

C:\Users\hboyz\AppData\Local\Temp\ipykernel_4576\1129610150.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.loc[key, "HTS6"] = values.HTS6.replace(".","").zfill(6)
C:\Users\hboyz\AppData\Local\Temp\ipykernel_4576\1129610150.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.loc[key, "HTS_9"] = pre_num_9
C:\Users\hboyz\AppData\Local\Temp\ipykernel_4576\1129610150.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d

In [ ]:
pre_num_9 = "000"
for key, values in df_new.iterrows():
    if values.HTS_9 == '000':
        print(values.HTS_9)
        pre_num_9 = values.HTS_9
    elif values.HTS_9 != '':     
        print(values.HTS_9)
        pre_num_9 = values.HTS_9
    else:
        print(pre_num_9)

In [ ]:
for key, values in df_new.iterrows():
    print(values.DESCRIPTION)

In [107]:
df_new.to_csv("japan_hts_all_digits.csv", index=False)